# LLMRouter Quick Start Tutorial

**Estimated Time:** 15 minutes  
**Level:** Beginner  
**Prerequisites:** None

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ulab-uiuc/LLMRouter/blob/main/tutorials/notebooks/00_Quick_Start.ipynb)

## Learning Objectives

By the end of this tutorial, you will:
- ✅ Install LLMRouter in Google Colab
- ✅ Understand basic routing concepts
- ✅ Run your first inference
- ✅ Try different built-in routers

---

## 1. Installation

First, let's install LLMRouter from GitHub.

In [ ]:
# Clone the repository
!git clone https://github.com/ulab-uiuc/LLMRouter.git
%cd LLMRouter

# Install the package
!pip install -e . -q

print("✅ Installation complete!")

## 2. Understanding LLMRouter

**What is LLMRouter?**

LLMRouter is an intelligent system that automatically selects the best LLM for each query based on:
- **Task complexity**: Simple queries → smaller models, complex queries → larger models
- **Cost optimization**: Balance performance and cost
- **Performance requirements**: Speed vs accuracy trade-offs

**Key Concepts:**

1. **Router**: Algorithm that decides which LLM to use
2. **Query**: Your input question/prompt
3. **LLM Candidates**: Pool of available models to choose from
4. **Routing Decision**: Which model is selected and why

## 3. List Available Routers

Let's see what routers are available out of the box.

In [ ]:
!llmrouter list-routers

## 4. Your First Routing - Route Only

Let's use a simple router (SmallestLLM) to make a routing decision without actually calling any LLM API.

This is useful for:
- Testing routing logic
- Understanding router behavior
- Avoiding API costs during development

In [ ]:
# Route a simple query
!llmrouter infer \
  --router smallest_llm \
  --config configs/model_config_test/smallest_llm.yaml \
  --query "What is 2+2?" \
  --route-only \
  --verbose

**Understanding the Output:**

The router returns:
- `model_name`: Which LLM was selected
- `query`: Your original query
- Additional metadata (depends on router type)

## 5. Try Different Routers

Let's compare how different routers make decisions for the same query.

In [ ]:
# List of routers to try
routers = [
    ('smallest_llm', 'configs/model_config_test/smallest_llm.yaml'),
    ('largest_llm', 'configs/model_config_test/largest_llm.yaml'),
    ('knnrouter', 'configs/model_config_test/knnrouter.yaml'),
]

query = "Explain quantum computing in simple terms"

print(f"Query: {query}\n")
print("="*60)

for router_name, config_path in routers:
    print(f"\n🔍 Router: {router_name}")
    !llmrouter infer \
      --router {router_name} \
      --config {config_path} \
      --query "{query}" \
      --route-only \
      --verbose 2>&1 | grep -E "(Routed to|model_name)"
    print("-"*60)

## 6. Batch Inference (Multiple Queries)

Let's route multiple queries at once.

In [ ]:
# Create a file with multiple queries
queries = [
    "What is 2+2?",
    "Explain quantum computing",
    "Write a Python function to sort a list",
    "Translate 'hello' to Spanish",
    "What is the meaning of life?"
]

# Save to file
with open('test_queries.txt', 'w') as f:
    for q in queries:
        f.write(q + '\n')

print("✅ Created test_queries.txt with", len(queries), "queries")

In [ ]:
# Route all queries
!llmrouter infer \
  --router knnrouter \
  --config configs/model_config_test/knnrouter.yaml \
  --input test_queries.txt \
  --output results.json \
  --route-only \
  --verbose

In [ ]:
# View results
import json

with open('results.json', 'r') as f:
    results = json.load(f)

# Pretty print results
for i, result in enumerate(results, 1):
    print(f"\n{i}. Query: {result['query'][:50]}...")
    print(f"   → Routed to: {result.get('model_name', 'N/A')}")
    print(f"   Success: {result['success']}")

## 7. Understanding Configuration Files

Let's examine what's in a router configuration file.

In [ ]:
# View a config file
!cat configs/model_config_test/knnrouter.yaml

**Key Configuration Sections:**

1. **`data_path`**: Paths to data files
   - `llm_data`: Available LLM models
   - `query_data_test`: Test queries
   - `routing_data_test`: Ground truth routing labels

2. **`model_path`**: Where to save/load trained models

3. **`metric.weights`**: How to evaluate routing decisions
   - Performance, cost, LLM judge scores

4. **`hparam`**: Router-specific hyperparameters

## 8. Examining LLM Candidates

Let's see what LLM models are available.

In [ ]:
import json

# Load LLM data
with open('data/example_data/llm_candidates/default_llm.json', 'r') as f:
    llm_data = json.load(f)

print(f"Total LLM models available: {len(llm_data)}\n")
print("="*60)

for model_name, model_info in llm_data.items():
    print(f"\n📦 {model_name}")
    print(f"   Model: {model_info.get('model', 'N/A')}")
    print(f"   Size: {model_info.get('size', 'N/A')}")
    if 'cost' in model_info:
        print(f"   Cost: ${model_info['cost']}/1K tokens")

## 9. Next Steps

🎉 Congratulations! You've completed the Quick Start tutorial!

### What You Learned:
- ✅ How to install LLMRouter
- ✅ Basic routing concepts
- ✅ Using different routers
- ✅ Batch inference
- ✅ Configuration files

### Recommended Next Tutorials:

1. **[01_Installation_and_Setup.ipynb](01_Installation_and_Setup.ipynb)** - Deep dive into setup
2. **[02_Data_Preparation.ipynb](02_Data_Preparation.ipynb)** - Understanding data formats
3. **[03_Training_Single_Round_Routers.ipynb](03_Training_Single_Round_Routers.ipynb)** - Train your first router

### Additional Resources:
- 📖 [Main Documentation](https://github.com/ulab-uiuc/LLMRouter)
- 💬 [Slack Community](https://join.slack.com/t/llmrouteropen-ri04588/shared_invite/...)
- 🐛 [GitHub Issues](https://github.com/ulab-uiuc/LLMRouter/issues)

## 10. Exercises (Optional)

Try these to reinforce your learning:

**Exercise 1:** Create your own list of 10 queries and route them using 3 different routers. Compare the results.

**Exercise 2:** Examine the `data/example_data/routing_data/default_routing_test_data.jsonl` file. What information does it contain?

**Exercise 3:** Modify a configuration file to change the metric weights. How does this affect routing decisions?